TclError: no display name and no $DISPLAY environment variable

In [2]:
!pip install gradio
import gradio as gr
import json
import string

# --- šifrovací logika ---
abc = list(string.ascii_lowercase)
sym = list(":;!@#$%^&*()-_=+[]{}<>?/|,.~`^")[:26]

p2o_map = {p: o for p, o in zip(abc, sym)}
o2p_map = {o: p for p, o in zip(abc, sym)}

def p2o(text):
    return ''.join(p2o_map.get(ch, ch) for ch in text.lower())

def o2p(text):
    return ''.join(o2p_map.get(ch, ch) for ch in text)

def encode_json(data):
    if isinstance(data, dict):
        return {encode_json(k): encode_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [encode_json(i) for i in data]
    elif isinstance(data, str):
        return p2o(data)
    else:
        return data

def decode_json(data):
    if isinstance(data, dict):
        return {decode_json(k): decode_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [decode_json(i) for i in data]
    elif isinstance(data, str):
        return o2p(data)
    else:
        return data

# --- funkce pro Gradio ---
def process_json(json_text, action):
    try:
        data = json.loads(json_text)
    except Exception as e:
        return f"Chyba při načítání JSON: {e}"

    if action == "Šifrovat":
        result = encode_json(data)
    else:
        result = decode_json(data)
    return json.dumps(result, ensure_ascii=False, indent=2)

def process_text(text):
    enc = p2o(text)
    dec = o2p(enc)
    return enc, dec

# --- GUI ---
with gr.Blocks() as app:
    gr.Markdown("#  Šifrování JSON a textu")
    gr.Markdown("Jednoduchá aplikace na převod textu a JSON pomocí symbolů")

    with gr.Tab("Text"):
        vstup = gr.Textbox(label="Zadej text", placeholder="Sem napiš text…")
        vystup_enc = gr.Textbox(label="Zašifrovaný text")
        vystup_dec = gr.Textbox(label="Dešifrovaný text")
        btn = gr.Button("Šifrovat")
        btn.click(process_text, inputs=vstup, outputs=[vystup_enc, vystup_dec])

    with gr.Tab("JSON"):
        json_in = gr.Textbox(lines=10, label="Vlož JSON")
        akce = gr.Radio(["Šifrovat", "Dešifrovat"], value="Šifrovat", label="Akce")
        json_out = gr.Textbox(lines=10, label="Výsledek")
        btn2 = gr.Button("Spustit")
        btn2.click(process_json, inputs=[json_in, akce], outputs=json_out)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://170c411a902b7e547f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
